# 6주차 과제
updated 1010  
(1) classification 알고리즘 중에서 SVM, Logistic, Random Forest 알고리즘 원리 등 요약 각 1페이지 내외
(2) 주어진 엑셀파일을 mysqld의 테이블로 구축하라.   
(3) grade B에 대한 3가지 알고리즘들의 이진분류 성능 결과를 얻는 파이썬 프로그램 작성, 결과를 제시/분석  
(4) grade A,B,C 에 대한 3가지 알고리즘의 다항분류 성능결과를 얻는 파이썬 프로그램 작성

In [32]:
# ready
import pandas as pd
import numpy as np

import pymysql
import sqlalchemy

import warnings
warnings.filterwarnings(action='ignore')

from IPython.display import display
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option('display.max_info_columns', 500)

# sklearn
from sklearn.model_selection import train_test_split,KFold
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

### (2) 주어진 데이터 파일을 mysql 에 테이블로 구축

In [33]:
# data
db_score3 = pd.read_excel('data/db_score_3_labels.xlsx')
db_score3.head(2)

,sno,homework,discussion,midterm,grade
0,1,19.37,0,35.00,A
1,2,20.00,0,34.12,A


####  python to database function

In [34]:
# python to database function
def make_data_table():
    # database 연결
    conn = pymysql.connect(host='localhost', user='root',
                           password='1234',db='university')
    curs = conn.cursor(pymysql.cursors.DictCursor)
    
    drop_sql = '''drop table if exists db_score3'''
    curs.execute(drop_sql)
    conn.commit()
    
    db_username = 'root'
    db_password = '1234'
    db_ip = 'localhost'
    db_name = 'university'
    db_connection = sqlalchemy.create_engine('mysql+pymysql://{0}:{1}@{2}/{3}'.format(db_username,db_password,
                                                                                     db_ip,db_name))
    db_score3.to_sql(con=db_connection, name='db_score3', if_exists='replace')  
    
make_data_table()

####  db to python

In [51]:
# db to python
conn = pymysql.connect(host='localhost', user='root', password='1234', db='university')
curs = conn.cursor(pymysql.cursors.DictCursor)

sql = '''
    select * from db_score3
    '''
curs.execute(sql)
# data
data = curs.fetchall()
print(data)

# close
curs.close()
conn.close()

[{'index': 0, 'sno': 1, 'homework': 19.37, 'discussion': 0, 'midterm': 35.0, 'grade': 'A'}, {'index': 1, 'sno': 2, 'homework': 20.0, 'discussion': 0, 'midterm': 34.12, 'grade': 'A'}, {'index': 2, 'sno': 3, 'homework': 19.37, 'discussion': 0, 'midterm': 34.3, 'grade': 'A'}, {'index': 3, 'sno': 4, 'homework': 18.12, 'discussion': 2, 'midterm': 31.5, 'grade': 'A'}, {'index': 4, 'sno': 5, 'homework': 20.0, 'discussion': 2, 'midterm': 33.25, 'grade': 'A'}, {'index': 5, 'sno': 6, 'homework': 19.37, 'discussion': 0, 'midterm': 24.5, 'grade': 'A'}, {'index': 6, 'sno': 7, 'homework': 19.37, 'discussion': 2, 'midterm': 23.8, 'grade': 'A'}, {'index': 7, 'sno': 8, 'homework': 16.87, 'discussion': 0, 'midterm': 28.87, 'grade': 'A'}, {'index': 8, 'sno': 9, 'homework': 19.37, 'discussion': 0, 'midterm': 29.75, 'grade': 'A'}, {'index': 9, 'sno': 10, 'homework': 20.0, 'discussion': 0, 'midterm': 31.5, 'grade': 'A'}, {'index': 10, 'sno': 11, 'homework': 19.37, 'discussion': 0, 'midterm': 29.57, 'grade':

#### Data Generation

In [52]:
X = [(d['index'],d['sno'],d['homework'],d['discussion'],d['midterm']) for d in data]
X = np.array(X)

y = [1 if (d['grade']=='B') else -1 for d in data]
y = np.array(y)
 
X[:3] , y[:3]

(array([[ 0.  ,  1.  , 19.37,  0.  , 35.  ],
        [ 1.  ,  2.  , 20.  ,  0.  , 34.12],
        [ 2.  ,  3.  , 19.37,  0.  , 34.3 ]]),
 array([-1, -1, -1]))

In [53]:
import collections
collections.Counter(y)

Counter({-1: 61, 1: 31})

### (3) grade B에 대한 3가지 알고리즘들의 이진분류 성능 결과를 얻는 파이썬 프로그램 작성, 결과를 제시/분석
- train, test split 과 kfold 사용
- 4가지 성능평가

#### setting

In [54]:
# train_test_split
# average='micro', 'binary'
def split_eval(model,X,y,average='binary'):
    # split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    clf = model.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    
    # eval
    acc = accuracy_score(y_test,y_pred)
    pre = precision_score(y_test,y_pred,average=average)
    rec = recall_score(y_test,y_pred,average=average)
    f1 = f1_score(y_test,y_pred,average=average)

    # print
    print(f'Accracy : {acc}')
    print(f'Precision : {pre}')
    print(f'Recall : {rec}')
    print(f'F1 : {f1}')
    
    return acc,pre,rec,f1

# K-Fold (cv=5, 5-fold 교차검증)
def kf_eval(model,X,y,cv=5,average='binary'):
    # kf setting
    kf = KFold(n_splits=cv, random_state=42, shuffle=True)
    accs = []
    pres = []
    recs = []
    f1s = []

    for t,v in kf.split(X):
        X_train,X_val = X[t],X[v]
        y_train,y_val = y[t],y[v]

        # model 
        clf = model.fit(X_train,y_train)
        y_pred = clf.predict(X_val)

        # eval
        acc = accuracy_score(y_val,y_pred)
        pre = precision_score(y_val,y_pred,average=average)
        rec = recall_score(y_val,y_pred,average=average)
        f1 = f1_score(y_val,y_pred,average=average)
        
        # print
        print(f'Accracy : {acc}')
        print(f'Precision : {pre}')
        print(f'Recall : {rec}')
        print(f'F1 : {f1}')
        print(''' ''')
        # save
        accs.append(acc)
        pres.append(pre)
        recs.append(rec)
        f1s.append(f1)

    # mean
    print(f'Total Accracy : {np.mean(accs)}')
    print(f'Total Precision : {np.mean(pres)}')
    print(f'Total Recall : {np.mean(recs)}')
    print(f'Total F1_score : {np.mean(f1s)}')
    return np.mean(accs),np.mean(pres),np.mean(recs),np.mean(f1s)

#### Classification & Evaluate

In [55]:
# model setting
svm = SVC()
lr = LogisticRegression()
rf = RandomForestClassifier()

* svm

In [56]:
svm_acc, svm_pre, svm_rec, svm_f1 = split_eval(svm,X,y)
print('-'*50)
svm_kf_acc, svm_kf_pre, svm_kf_rec, svm_kf_f1 = kf_eval(svm,X,y,cv=5)

Accracy : 0.9642857142857143
Precision : 0.8888888888888888
Recall : 1.0
F1 : 0.9411764705882353
--------------------------------------------------
Accracy : 0.8947368421052632
Precision : 0.75
Recall : 1.0
F1 : 0.8571428571428571
 
Accracy : 0.9473684210526315
Precision : 1.0
Recall : 0.8571428571428571
F1 : 0.923076923076923
 
Accracy : 1.0
Precision : 1.0
Recall : 1.0
F1 : 1.0
 
Accracy : 1.0
Precision : 1.0
Recall : 1.0
F1 : 1.0
 
Accracy : 0.9444444444444444
Precision : 0.8333333333333334
Recall : 1.0
F1 : 0.9090909090909091
 
Total Accracy : 0.9573099415204679
Total Precision : 0.9166666666666666
Total Recall : 0.9714285714285715
Total F1_score : 0.9378621378621379


* Logistic Regression

In [57]:
lr_acc, lr_pre, lr_rec, lr_f1 = split_eval(lr,X,y)
print('-'*50)
lr_kf_acc, lr_kf_pre, lr_kf_rec, lr_kf_f1 = kf_eval(lr,X,y)

Accracy : 0.7142857142857143
Precision : 0.5
Recall : 0.125
F1 : 0.2
--------------------------------------------------
Accracy : 0.6842105263157895
Precision : 0.5
Recall : 0.16666666666666666
F1 : 0.25
 
Accracy : 0.631578947368421
Precision : 0.0
Recall : 0.0
F1 : 0.0
 
Accracy : 0.7222222222222222
Precision : 0.25
Recall : 0.3333333333333333
F1 : 0.28571428571428575
 
Accracy : 0.5
Precision : 1.0
Recall : 0.1
F1 : 0.18181818181818182
 
Accracy : 0.5
Precision : 0.16666666666666666
Recall : 0.2
F1 : 0.1818181818181818
 
Total Accracy : 0.6076023391812866
Total Precision : 0.38333333333333336
Total Recall : 0.16
Total F1_score : 0.1798701298701299


In [58]:
rf_acc, rf_pre, rf_rec, rf_f1 = split_eval(rf,X,y)
print('-'*50)
rf_kf_acc, rf_kf_pre, rf_kf_rec, rf_kf_f1 = kf_eval(rf,X,y,cv=5)

Accracy : 0.9642857142857143
Precision : 1.0
Recall : 0.875
F1 : 0.9333333333333333
--------------------------------------------------
Accracy : 1.0
Precision : 1.0
Recall : 1.0
F1 : 1.0
 
Accracy : 0.9473684210526315
Precision : 1.0
Recall : 0.8571428571428571
F1 : 0.923076923076923
 
Accracy : 1.0
Precision : 1.0
Recall : 1.0
F1 : 1.0
 
Accracy : 0.9444444444444444
Precision : 0.9090909090909091
Recall : 1.0
F1 : 0.9523809523809523
 
Accracy : 0.9444444444444444
Precision : 1.0
Recall : 0.8
F1 : 0.888888888888889
 
Total Accracy : 0.967251461988304
Total Precision : 0.9818181818181818
Total Recall : 0.9314285714285715
Total F1_score : 0.9528693528693528


In [59]:
svm_split_eval = [svm_acc, svm_pre, svm_rec, svm_f1]
svm_kf_eval = [svm_kf_acc, svm_kf_pre, svm_kf_rec, svm_kf_f1]
lr_split_eval = [lr_acc, lr_pre, lr_rec, lr_f1]
lr_kf_eval = [lr_kf_acc, lr_kf_pre, lr_kf_rec, lr_kf_f1]
rf_split_eval = [rf_acc, rf_pre, rf_rec, rf_f1]
rf_kf_eval = [rf_kf_acc, rf_kf_pre, rf_kf_rec, rf_kf_f1]

# result
bi_result = pd.DataFrame(index=['Accuracy','Precision','Recall','F1_score'])
bi_result['svm_split_eval'] = svm_split_eval
bi_result['svm_kf_eval'] = svm_kf_eval
bi_result['lr_split'] = lr_split_eval
bi_result['lr_kf_eval'] = lr_kf_eval
bi_result['rf_split_eval'] = rf_split_eval
bi_result['rf_kf_eval'] = rf_kf_eval

bi_result

,svm_split_eval,svm_kf_eval,lr_split,lr_kf_eval,rf_split_eval,rf_kf_eval
Accuracy,0.964286,0.957310,0.714286,0.607602,0.964286,0.967251
Precision,0.888889,0.916667,0.500000,0.383333,1.000000,0.981818
Recall,1.000000,0.971429,0.125000,0.160000,0.875000,0.931429
F1_score,0.941176,0.937862,0.200000,0.179870,0.933333,0.952869


### (4) grade A,B,C 에 대한 3가지 알고리즘의 다항분류 성능결과를 얻는 파이썬 프로그램 작성
다항분류이기 때문에 y를 숫자로 라벨링 해준다.

In [60]:
# load y
y = [d['grade'] for d in data ]
le = preprocessing.LabelEncoder()
y_label = le.fit_transform(y)
y_label

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2])

In [61]:
svm_acc, svm_pre, svm_rec, svm_f1 = split_eval(svm,X,y_label,average='micro')
print('-'*50)
svm_kf_acc, svm_kf_pre, svm_kf_rec, svm_kf_f1 = kf_eval(svm,X,y_label,cv=5,average='micro')

Accracy : 0.9285714285714286
Precision : 0.9285714285714286
Recall : 0.9285714285714286
F1 : 0.9285714285714286
--------------------------------------------------
Accracy : 0.9473684210526315
Precision : 0.9473684210526315
Recall : 0.9473684210526315
F1 : 0.9473684210526315
 
Accracy : 0.8947368421052632
Precision : 0.8947368421052632
Recall : 0.8947368421052632
F1 : 0.8947368421052632
 
Accracy : 1.0
Precision : 1.0
Recall : 1.0
F1 : 1.0
 
Accracy : 1.0
Precision : 1.0
Recall : 1.0
F1 : 1.0
 
Accracy : 0.8888888888888888
Precision : 0.8888888888888888
Recall : 0.8888888888888888
F1 : 0.8888888888888888
 
Total Accracy : 0.9461988304093566
Total Precision : 0.9461988304093566
Total Recall : 0.9461988304093566
Total F1_score : 0.9461988304093566


In [62]:
lr_acc, lr_pre, lr_rec, lr_f1 = split_eval(lr,X,y_label,average='micro')
print('-'*50)
lr_kf_acc, lr_kf_pre, lr_kf_rec, lr_kf_f1 = kf_eval(lr,X,y_label,cv=5,average='micro')

Accracy : 0.9285714285714286
Precision : 0.9285714285714286
Recall : 0.9285714285714286
F1 : 0.9285714285714286
--------------------------------------------------
Accracy : 0.8947368421052632
Precision : 0.8947368421052632
Recall : 0.8947368421052632
F1 : 0.8947368421052632
 
Accracy : 0.8947368421052632
Precision : 0.8947368421052632
Recall : 0.8947368421052632
F1 : 0.8947368421052632
 
Accracy : 1.0
Precision : 1.0
Recall : 1.0
F1 : 1.0
 
Accracy : 0.9444444444444444
Precision : 0.9444444444444444
Recall : 0.9444444444444444
F1 : 0.9444444444444444
 
Accracy : 0.8333333333333334
Precision : 0.8333333333333334
Recall : 0.8333333333333334
F1 : 0.8333333333333334
 
Total Accracy : 0.9134502923976608
Total Precision : 0.9134502923976608
Total Recall : 0.9134502923976608
Total F1_score : 0.9134502923976608


In [63]:
rf_acc, rf_pre, rf_rec, rf_f1 = split_eval(rf,X,y_label,average='micro')
print('-'*50)
rf_kf_acc, rf_kf_pre, rf_kf_rec, rf_kf_f1 = kf_eval(rf,X,y_label,cv=5,average='micro')

Accracy : 0.9642857142857143
Precision : 0.9642857142857143
Recall : 0.9642857142857143
F1 : 0.9642857142857143
--------------------------------------------------
Accracy : 1.0
Precision : 1.0
Recall : 1.0
F1 : 1.0
 
Accracy : 0.9473684210526315
Precision : 0.9473684210526315
Recall : 0.9473684210526315
F1 : 0.9473684210526315
 
Accracy : 1.0
Precision : 1.0
Recall : 1.0
F1 : 1.0
 
Accracy : 0.9444444444444444
Precision : 0.9444444444444444
Recall : 0.9444444444444444
F1 : 0.9444444444444444
 
Accracy : 0.9444444444444444
Precision : 0.9444444444444444
Recall : 0.9444444444444444
F1 : 0.9444444444444444
 
Total Accracy : 0.967251461988304
Total Precision : 0.967251461988304
Total Recall : 0.967251461988304
Total F1_score : 0.967251461988304


In [64]:
svm_split_eval = [svm_acc, svm_pre, svm_rec, svm_f1]
svm_kf_eval = [svm_kf_acc, svm_kf_pre, svm_kf_rec, svm_kf_f1]
lr_split_eval = [lr_acc, lr_pre, lr_rec, lr_f1]
lr_kf_eval = [lr_kf_acc, lr_kf_pre, lr_kf_rec, lr_kf_f1]
rf_split_eval = [rf_acc, rf_pre, rf_rec, rf_f1]
rf_kf_eval = [rf_kf_acc, rf_kf_pre, rf_kf_rec, rf_kf_f1]

# result
multi_result = pd.DataFrame(index=['Accuracy','Precision','Recall','F1_score'])
multi_result['svm_split_eval'] = svm_split_eval
multi_result['svm_kf_eval'] = svm_kf_eval
multi_result['lr_split'] = lr_split_eval
multi_result['lr_kf_eval'] = lr_kf_eval
multi_result['rf_split_eval'] = rf_split_eval
multi_result['rf_kf_eval'] = rf_kf_eval

multi_result

,svm_split_eval,svm_kf_eval,lr_split,lr_kf_eval,rf_split_eval,rf_kf_eval
Accuracy,0.928571,0.946199,0.928571,0.91345,0.964286,0.967251
Precision,0.928571,0.946199,0.928571,0.91345,0.964286,0.967251
Recall,0.928571,0.946199,0.928571,0.91345,0.964286,0.967251
F1_score,0.928571,0.946199,0.928571,0.91345,0.964286,0.967251


In [65]:
bi_result

,svm_split_eval,svm_kf_eval,lr_split,lr_kf_eval,rf_split_eval,rf_kf_eval
Accuracy,0.964286,0.957310,0.714286,0.607602,0.964286,0.967251
Precision,0.888889,0.916667,0.500000,0.383333,1.000000,0.981818
Recall,1.000000,0.971429,0.125000,0.160000,0.875000,0.931429
F1_score,0.941176,0.937862,0.200000,0.179870,0.933333,0.952869
